In [1]:
import numpy as np
import keras
import keras.backend as k
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import image
import cv2
import datetime

In [2]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-08-01 12:34:12.475915: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-01 12:34:12.476489: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'train',
        target_size=(150,150),
        batch_size=16 ,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'test',
        target_size=(150,150),
        batch_size=16,
        class_mode='binary')

model_saved=model.fit_generator(
        training_set,
        epochs=10,
        validation_data=test_set,

        )

model.save('mymodel.h5',model_saved)

Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.
Epoch 1/10


/var/folders/yf/50hlh2px1dddy3cwqth2cj5r0000gn/T/ipykernel_43884/2366956476.py:22: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_saved=model.fit_generator(
2022-08-01 12:34:12.861125: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-01 12:34:13.054199: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


83/83 [==============================] - ETA: 0s - loss: 0.2770 - accuracy: 0.8706

2022-08-01 12:34:19.902419: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


83/83 [==============================] - 7s 84ms/step - loss: 0.2770 - accuracy: 0.8706 - val_loss: 0.1372 - val_accuracy: 0.9536
Epoch 2/10
83/83 [==============================] - 7s 80ms/step - loss: 0.1243 - accuracy: 0.9589 - val_loss: 0.0965 - val_accuracy: 0.9639
Epoch 3/10
83/83 [==============================] - 6s 77ms/step - loss: 0.1046 - accuracy: 0.9642 - val_loss: 0.1024 - val_accuracy: 0.9639
Epoch 4/10
83/83 [==============================] - 7s 78ms/step - loss: 0.0942 - accuracy: 0.9658 - val_loss: 0.0760 - val_accuracy: 0.9845
Epoch 5/10
83/83 [==============================] - 7s 78ms/step - loss: 0.0827 - accuracy: 0.9688 - val_loss: 0.0846 - val_accuracy: 0.9742
Epoch 6/10
83/83 [==============================] - 6s 76ms/step - loss: 0.0873 - accuracy: 0.9711 - val_loss: 0.0925 - val_accuracy: 0.9588
Epoch 7/10
83/83 [==============================] - 6s 75ms/step - loss: 0.0683 - accuracy: 0.9795 - val_loss: 0.0625 - val_accuracy: 0.9742
Epoch 8/10
83/83 [======

In [4]:
# mymodel=load_model('mymodel.h5')

# test_image=image.load_img(r'/Users/adityamishra/Desktop/Screenshot 2022-03-16 at 11.08.07 AM.png',
#                           target_size=(150,150,3))
# test_image
# test_image=image.img_to_array(test_image)
# test_image=np.expand_dims(test_image,axis=0)
# mymodel.predict(test_image)[0][0]

In [6]:
mymodel=load_model('mymodel.h5')

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

2022-08-01 12:37:39.106709: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
